In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms, models
from torchsummary import summary

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
import pandas as pd

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Colab Projects/BioDiversity Project

/content/drive/My Drive/Colab Projects/BioDiversity Project


In [ ]:
df = pd.read_csv(r"/content/drive/MyDrive/Colab Projects/BioDiversity Project/data/updated_species_data.csv")

In [ ]:
import os

# Define the path to your dataset
data_dir = 'species'  # Change this to the correct path

# Get the label names (folder names) dynamically from the directory structure
labels = [label for label in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, label))]

# Print the extracted labels
print(labels)

len(labels)


['Tupaia belangeri', 'Uncia uncia', 'Nilgiritragus hylocrius', 'Macaca silenus', 'Tupaia nicobarica', 'Anathana ellioti', 'Prionailurus bengalensis', 'Hyaena hyaena', 'Lutrogale perspicillata', 'Catopuma temminckii', 'Pardofelis marmorata', 'Panthera pardus', 'Aonyx cinereus', 'Bos frontalis', 'Herpestes edwardsi', 'Macaca radiata', 'Ratufa macroura', 'Pteropus giganteus', 'Rhinolophus lepidus', 'Bubalus bubalis', 'Elephas maximus', 'Felis catus', 'Callosciurus pygerythrus', 'Lepus nigricollis', 'Ratufa bicolor', 'Hoolock hoolock', 'Trachypithecus pileatus', 'Macaca mulatta', 'Funambulus sublineatus', 'Rusa unicolor', 'Boselaphus tragocamelus', 'Sus scrofa', 'Funambulus palmarum', 'Canis aureus', 'Gazella bennettii', 'Paguma larvata', 'Paraechinus micropus', 'Bos grunniens', 'Trachypithecus phayrei', 'Axis axis', 'Paradoxurus hermaphroditus', 'Callosciurus erythraeus', 'Felis chaus', 'Antilope cervicapra', 'Muntiacus muntjak', 'Semnopithecus entellus', 'Nycticebus bengalensis', 'Parasc

83

In [ ]:
# Extract unique species names from the 'SpeciesName' column
unique_species = df['SpeciesName'].unique()

# Create labels using the unique species names
labels = list(unique_species[:200])

print(labels)

In [ ]:
import os
import shutil

# List of folders to delete
folders_to_delete = [
    "Manis crassicaudata",
    "Urva urva",
    "Petinomys fuscocapillus",
    "Rousettus leschenaultii",
    "Megaderma lyra",
    "Megaderma spasma",
    "Cannomys badius",
    "Martes gwatkinsii",
    "Trachypithecus geei",
    "Petaurista philippensis",
    "Herpestes urva",
    "Hemitragus jemlahicus",
    "Dremomys lokriah",
    "Suncus murinus",
    "Sousa plumbea",
    "Rhinoceros sondaicus",
    "Pseudois nayaur"
]

# Define the path to the "species" directory
species_dir = 'species'  # Change this to the correct path

# Loop through the folders to delete and remove them
for folder in folders_to_delete:
    folder_path = os.path.join(species_dir, folder)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Deleted folder: {folder_path}")
    else:
        print(f"Folder not found: {folder_path}")


Folder not found: species/Manis crassicaudata
Folder not found: species/Urva urva
Folder not found: species/Petinomys fuscocapillus
Folder not found: species/Rousettus leschenaultii
Folder not found: species/Megaderma lyra
Folder not found: species/Megaderma spasma
Folder not found: species/Cannomys badius
Folder not found: species/Martes gwatkinsii
Folder not found: species/Trachypithecus geei
Folder not found: species/Petaurista philippensis
Folder not found: species/Herpestes urva
Folder not found: species/Hemitragus jemlahicus
Folder not found: species/Dremomys lokriah
Folder not found: species/Suncus murinus
Folder not found: species/Sousa plumbea
Folder not found: species/Rhinoceros sondaicus
Folder not found: species/Pseudois nayaur


In [ ]:
data_path = "species"

# Loop through the first 100 species labels
for label in labels:
    species_dir = os.path.join(data_path, label)
    images_dir = os.path.join(species_dir, "images")
    train_dir = os.path.join(images_dir, "train")
    valid_dir = os.path.join(images_dir, "valid")

    # Create the necessary folders if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(valid_dir, exist_ok=True)

print("Folders created successfully.")

Folders created successfully.


In [ ]:
import os
import random
import shutil

data_path = "species"

# Define the proportion of images to keep in the training set (e.g., 80%)
train_proportion = 0.8

# Loop through the first 100 species labels
for label in labels:
    species_dir = os.path.join(data_path, label)
    images_dir = os.path.join(species_dir, "images")

    # Create train and valid folders if they don't exist
    train_dir = os.path.join(images_dir, "train")
    valid_dir = os.path.join(images_dir, "valid")
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(valid_dir, exist_ok=True)

    # List all image files in the species' image folder
    image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]

    # Shuffle the list of image files
    random.shuffle(image_files)

    # Determine the number of images for the training set
    num_train = int(len(image_files) * train_proportion)

    # Copy images to train and valid folders
    for i, image_file in enumerate(image_files):
        source_path = os.path.join(images_dir, image_file)
        if i < num_train:
            target_path = os.path.join(train_dir, image_file)
        else:
            target_path = os.path.join(valid_dir, image_file)
        shutil.move(source_path, target_path)

print("Images copied into train and valid sets successfully.")

Images copied into train and valid sets successfully.


In [ ]:
import os
import shutil

data_path = "species"

# Loop through the first 100 species labels
for label in labels:
    species_dir = os.path.join(data_path, label)
    images_dir = os.path.join(species_dir, "images")

    # Define the train and valid folders
    train_dir = os.path.join(images_dir, "train")
    valid_dir = os.path.join(images_dir, "valid")

    # List image files in train and valid folders
    train_image_files = [f for f in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, f))]
    valid_image_files = [f for f in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, f))]

    # Move images from train and valid folders to the main class folder
    for image_file in train_image_files:
        source_path = os.path.join(train_dir, image_file)
        target_path = os.path.join(images_dir, image_file)
        shutil.move(source_path, target_path)

    for image_file in valid_image_files:
        source_path = os.path.join(valid_dir, image_file)
        target_path = os.path.join(images_dir, image_file)
        shutil.move(source_path, target_path)

    # Remove train and valid folders
    os.rmdir(train_dir)
    os.rmdir(valid_dir)

print("Images moved back to class folders successfully.")


Images moved back to class folders successfully.


In [ ]:
import os
import shutil

data_path = "species"

# Get the label names (folder names) dynamically from the directory structure
labels = [label for label in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, label))]

# Loop through the labels
for label in labels:
    species_dir = os.path.join(data_path, label)
    images_dir = os.path.join(species_dir, "images")

    # List image files in the source directory (images)
    image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]

    # Move images to their respective class folders
    for image_file in image_files:
        # Create the target directory for the class label if it doesn't exist
        class_folder = os.path.join(data_path, label)
        os.makedirs(class_folder, exist_ok=True)

        # Define the source and target paths
        source_path = os.path.join(images_dir, image_file)
        target_path = os.path.join(class_folder, image_file)

        # Move the image file
        shutil.move(source_path, target_path)

    # Remove the now-empty "images" subfolder for this label
    os.rmdir(images_dir)

print("Images moved to class folders successfully.")


Images moved to class folders successfully.


In [ ]:
import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import UnidentifiedImageError  # Import the exception

# Define the path to your dataset
data_dir = 'species'  # Change this to the correct path

# Get the label names (folder names) dynamically from the loaded labels
labels = [label for label in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, label))]

# Define data transformations (customize as needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create ImageFolder datasets for training and validation for each label
train_datasets = []
valid_datasets = []

for label in labels:
    try:
        train_dataset = ImageFolder(os.path.join(data_dir), transform=transform)
        valid_dataset = ImageFolder(os.path.join(data_dir), transform=transform)
        train_datasets.append(train_dataset)
        valid_datasets.append(valid_dataset)
    except UnidentifiedImageError as e:
        print(f"Error for label '{label}': {str(e)}")
        continue  # Continue to the next label if there's an error

# Define batch size
batch_size = 32  # Adjust as needed

# Create DataLoader objects for training and validation for each label
train_loaders = [DataLoader(train_data, batch_size=bs, shuffle=True) for dataset in train_datasets]
valid_loaders = [DataLoader(dataset, batch_size=batch_size) for dataset in valid_datasets]

# Example: Loop through the dataloaders for each label
for label, train_loader in zip(labels, train_loaders):
    print(f"Label: {label}")
    for images, batch_labels in train_loader:
        print("Image batch shape:", images.shape)
        print("Label batch:", batch_labels)
        break


Label: Tupaia belangeri
Image batch shape: torch.Size([32, 3, 224, 224])
Label batch: tensor([14, 43, 32, 31, 48, 81, 59, 72, 19, 10, 43, 71, 26, 53, 75, 71, 33, 44,
        18, 45, 78, 70, 71, 82, 51, 72, 57, 34, 40, 14, 25, 20])
Label: Uncia uncia
Image batch shape: torch.Size([32, 3, 224, 224])
Label batch: tensor([17, 38, 73, 36,  6,  3, 25, 36,  1, 15,  6,  9,  6, 25, 82,  7, 16,  3,
        61, 26,  8, 22, 45, 58,  8, 20, 79, 82, 35, 62, 71, 36])
Label: Nilgiritragus hylocrius
Image batch shape: torch.Size([32, 3, 224, 224])
Label batch: tensor([ 8, 57, 42, 15, 41, 63, 82, 18, 27, 11, 57, 68, 73,  2, 19, 51, 25, 46,
        70, 76, 64, 73, 60, 62, 82, 45, 80, 64,  2, 17, 18, 34])
Label: Macaca silenus
Image batch shape: torch.Size([32, 3, 224, 224])
Label batch: tensor([ 1, 32, 11, 27, 46, 37, 38,  9, 13, 82, 26, 71, 44, 71, 54, 33, 16,  3,
        21, 47,  1,  2, 35, 47, 10, 30, 45, 69, 45, 62, 70, 22])
Label: Tupaia nicobarica
Image batch shape: torch.Size([32, 3, 224, 224])
La

KeyboardInterrupt: ignored

In [ ]:
import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Define the path to your dataset
data_dir = 'species'  # Change this to the correct path

# Define data transformations (customize as needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create an ImageFolder dataset for the entire "species" folder
dataset = ImageFolder(data_dir, transform=transform)

# Define batch size
batch_size = 32  # Adjust as needed

# Create a DataLoader object for the dataset
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Loop through the DataLoader
for images, labels in data_loader:
    print("Image batch shape:", images.shape)
    print("Label batch:", labels)
    break


Image batch shape: torch.Size([32, 3, 224, 224])
Label batch: tensor([64, 33, 75, 73, 32, 43, 62, 62, 70,  0,  0, 53, 31, 63, 42, 13, 30, 81,
        46, 63,  5, 52, 17, 29, 66, 16, 82, 51, 25, 37, 33, 37])


In [ ]:
import os

data_path = "species"

# Initialize a dictionary to store the count of downloaded images for each label
image_counts = {}

# Loop through each label and count the images in the 'images' folder
for label in labels:
    label_path = os.path.join(data_path, label)
    images_path = os.path.join(label_path, "images")

    # Check if the 'images' folder exists
    if os.path.exists(images_path):
        # Count the number of image files in the 'images' folder
        image_count = len([f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png'))])
        image_counts[label] = image_count

# Identify labels with less than 150 images downloaded
labels_with_less_than_150_images = [label for label, count in image_counts.items() if count < 150]

# Print the labels and their corresponding image counts
for label, count in image_counts.items():
    print(f"Label: {label}, Image Count: {count}")

# Print the labels with less than 150 images downloaded
print("\nLabels with less than 150 images downloaded:")
for label in labels_with_less_than_150_images:
    print(label)



Labels with less than 150 images downloaded:


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []  # Store paths to images
        self.labels = []  # Store labels

        # Assuming that your data directory has subdirectories with class labels
        self.class_labels = sorted(os.listdir(data_dir))

        # Load image paths and labels
        for label_idx, class_label in enumerate(self.class_labels):
            class_dir = os.path.join(data_dir, class_label)
            image_files = os.listdir(class_dir)
            for image_file in image_files:
                self.image_paths.append(os.path.join(class_dir, image_file))
                self.labels.append(label_idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]

        # Load the image using PIL
        image = np.array(Image.open(image_path))

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        # Convert image to a tensor
        image = transforms.ToTensor()(image)

        return image, label

# Define data transformations (customize as needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  # Converts image to tensor after transformations
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Example: Create DataLoader for your custom dataset
data_dir = 'species'  # Change to the correct path
custom_dataset = CustomDataset(data_dir, transform=transform)
data_loader = DataLoader(custom_dataset, batch_size=32, shuffle=True)

# Example: Iterate through the DataLoader to access images and labels as tensors
for images, labels in data_loader:
    print("Image batch shape:", images.shape)  # Shape of the batch of images
    print("Label batch:", labels)  # Batch of label indices
    break  # Exit the loop after the first batch


TypeError: ignored

In [ ]:
import os
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

class CustomImageDataset(Dataset):
    def __init__(self, data_dir, split='Train', transform=None):
        self.data_dir = data_dir
        self.split_dir = os.path.join(data_dir, 'images', split)
        self.labels = [label for label in os.listdir(self.split_dir) if os.path.isdir(os.path.join(self.split_dir, label))]
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        label_dir = os.path.join(self.split_dir, label)
        image_files = os.listdir(label_dir)
        images = []
        for image_file in image_files:
            image_path = os.path.join(label_dir, image_file)
            image = Image.open(image_path)
            if self.transform:
                image = self.transform(image)
            images.append(image)
        return images, label

# Define the path to your dataset
data_dir = 'species'  # Change this to the correct path

# Get the label names (folder names) dynamically from the directory structure
labels = [label for label in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, label))]

# Define data transformations (customize as needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create custom dataset objects for training and validation for each label
train_datasets = [CustomImageDataset(os.path.join(data_dir, label), split='train', transform=transform) for label in labels]
valid_datasets = [CustomImageDataset(os.path.join(data_dir, label), split='valid', transform=transform) for label in labels]

# Create DataLoader objects for each label
batch_size = 16  # Adjust as needed
train_loaders = []
valid_loaders = []

for label, train_dataset, valid_dataset in zip(labels, train_datasets, valid_datasets):
    try:
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
        train_loaders.append(train_loader)
        valid_loaders.append(valid_loader)
    except Exception as e:
        print(f"Error for label '{label}': {str(e)}")
        print(len(labels))

# Example: Loop through each label and print the first batch of training data
for label, train_loader in zip(labels, train_loaders):
    print(f"Label: {label}")
    for images, batch_labels in train_loader:
        print("Image batch shape:", images[0].shape)
        print("Label batch:", batch_labels[0])
        break


In [ ]:
import os
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

class CustomImageDataset(Dataset):
    def __init__(self, data_dir, split='Train', transform=None):
        self.data_dir = data_dir
        self.split_dir = os.path.join(data_dir, 'images', split)
        self.labels = [label for label in os.listdir(self.split_dir) if os.path.isdir(os.path.join(self.split_dir, label))]
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        label_dir = os.path.join(self.split_dir, label)
        image_files = os.listdir(label_dir)
        images = []
        for image_file in image_files:
            image_path = os.path.join(label_dir, image_file)
            image = Image.open(image_path)
            if self.transform:
                image = self.transform(image)
            images.append(image)
        return images, label

# Define the path to your dataset
data_dir = 'species'  # Change this to the correct path

# Create custom dataset objects for training and validation for each label
train_datasets = []
valid_datasets = []
problematic_labels = []

for label in os.listdir(data_dir):
    try:
        train_datasets.append(CustomImageDataset(os.path.join(data_dir, label), split='train', transform=transform))
        valid_datasets.append(CustomImageDataset(os.path.join(data_dir, label), split='valid', transform=transform))
    except Exception as e:
        problematic_labels.append(label)
        print(f"Error for label '{label}': {str(e)}")

# Print the labels that caused errors
if problematic_labels:
  print("Labels that caused errors:", problematic_labels)
else:
  print("Everything looks good")


Everything looks good


In [ ]:
train_datasets[0]

In [ ]:
import os
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

class CustomImageDataset(Dataset):
    def __init__(self, data_dir, split='Train', transform=None):
        self.data_dir = data_dir
        self.split_dir = os.path.join(data_dir, 'images', split)
        self.labels = os.listdir(self.split_dir)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        label_dir = os.path.join(self.split_dir, label)
        image_files = os.listdir(label_dir)
        images = []
        for image_file in image_files:
            image_path = os.path.join(label_dir, image_file)
            image = Image.open(image_path)
            if self.transform:
                image = self.transform(image)
            images.append(image)
        return images, label

# Define the path to your dataset
data_dir = 'species'  # Change this to the correct path

# Get the label names (folder names)
labels = [label for label in os.listdir(os.path.join(data_dir, 'images', 'train')) if os.path.isdir(os.path.join(data_dir, 'images', 'train', label))]

# Define data transformations (customize as needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create custom dataset objects for training and validation for each label
train_datasets = [CustomImageDataset(os.path.join(data_dir, 'images', 'train', label), split='train', transform=transform) for label in labels]
valid_datasets = [CustomImageDataset(os.path.join(data_dir, 'images', 'valid', label), split='valid', transform=transform) for label in labels]

# Create DataLoader objects for each label
batch_size = 32  # Adjust as needed
train_loaders = [DataLoader(dataset, batch_size=batch_size, shuffle=True) for dataset in train_datasets]
valid_loaders = [DataLoader(dataset, batch_size=batch_size) for dataset in valid_datasets]

# Example: Loop through each label and print the first batch of training data
for label, train_loader in zip(labels, train_loaders):
    print(f"Label: {label}")
    for images, labels in train_loader:
        print("Image batch shape:", images[0].shape)
        print("Label batch:", labels[0])
        break


In [ ]:
# Check the number of datasets that meet the criteria
print(f"Number of train datasets: {len(train_datasets)}")
print(f"Number of valid datasets: {len(valid_datasets)}")


Number of train datasets: 0
Number of valid datasets: 0


In [ ]:
import os

# Define the path to your dataset
data_dir = 'species'  # Change this to the correct path

# Iterate through the labels and print the contents of train and valid directories
for label in labels:
    train_dir = os.path.join(data_dir, label, 'images', 'train')
    valid_dir = os.path.join(data_dir, label, 'images', 'valid')

    print(f"Label: {label}")

    if os.path.exists(train_dir):
        train_files = os.listdir(train_dir)
        print(f"Train Directory ({len(train_files)} files):")
        print(train_files)
    else:
        print("Train Directory does not exist.")

    if os.path.exists(valid_dir):
        valid_files = os.listdir(valid_dir)
        print(f"Valid Directory ({len(valid_files)} files):")
        print(valid_files)
    else:
        print("Valid Directory does not exist.")


Label: Tupaia belangeri
Train Directory (132 files):
['7.jpg', '199.jpg', '2.jpg', '122.jpg', '116.jpg', '24.jpg', '161.jpg', '171.jpg', '159.jpg', '47.jpg', '74.jpg', '60.jpg', '140.jpg', '108.jpg', '59.jpg', '5.jpg', '172.jpg', '200.jpg', '188.jpg', '136.jpg', '69.jpg', '115.jpg', '131.jpg', '13.jpg', '89.jpg', '45.jpg', '114.jpg', '71.jpg', '57.jpg', '121.jpg', '53.jpg', '139.jpg', '34.jpg', '98.jpg', '123.jpg', '137.jpg', '187.jpg', '68.jpg', '160.jpg', '124.jpg', '181.jpg', '78.jpg', '92.jpg', '95.jpg', '135.jpg', '64.jpg', '166.jpg', '184.jpg', '90.jpg', '36.jpg', '133.jpg', '103.jpg', '162.jpg', '145.jpg', '87.jpg', '192.jpg', '65.jpg', '17.jpg', '180.jpg', '21.jpg', '70.jpg', '14.jpg', '27.jpg', '99.jpg', '118.jpg', '126.jpg', '129.jpg', '100.jpg', '67.jpg', '111.jpg', '15.jpg', '113.jpg', '155.jpg', '73.jpg', '191.jpg', '104.jpg', '149.jpg', '165.jpg', '132.jpg', '167.jpg', '52.jpg', '158.jpg', '63.jpg', '94.jpg', '62.jpg', '156.jpg', '112.jpg', '144.jpg', '197.jpg', '42.jpg',